In [1]:
import numpy as np
import pandas as pd
import time
import yfinance as yf
import matplotlib.pyplot as plt
import functions as f

%load_ext autoreload
%autoreload 2

In [2]:
companies = f.get_companies()

### We filter companies by market cap and average volume

In [3]:
filtered_companies = f.filter_all_companies(companies=companies)

In [4]:
last_row_df = filtered_companies.tail(1).T
last_row_df.reset_index(inplace=True)
last_row_df.columns = ['Company', 'Last Value']
ticker_prices = last_row_df.set_index('Company')['Last Value'].round(2).to_dict()
last_row_df.set_index('Company', inplace=True)

In [14]:
tickers = filtered_companies.columns
option_chains = {}

for ticker in tickers:
    try:
        # Call the function for each ticker
        calls_all, puts_all, spot_price = f.get_option_chains_spot(ticker_symbol=ticker)
        
        # Store the result in the dictionary
        option_chains[ticker] = {
            'calls': calls_all,
            'puts': puts_all,
            'spot_price': spot_price
        }
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")

In [15]:
option_chains

{'AMD': {'calls':      strike  lastPrice  impliedVolatility  expiration  time_to_expiration
  0      70.0      67.55           0.000010  2024-11-29            0.019178
  1      75.0      65.65           0.000010  2024-11-29            0.019178
  2      80.0      57.27           0.000010  2024-11-29            0.019178
  3      85.0      52.24           0.000010  2024-11-29            0.019178
  4      90.0      47.28           0.000010  2024-11-29            0.019178
  ..      ...        ...                ...         ...                 ...
  898   270.0      12.50           0.062509  2027-01-15            2.147945
  899   280.0      11.40           0.125009  2027-01-15            2.147945
  900   290.0      10.38           0.125009  2027-01-15            2.147945
  901   300.0       9.78           0.125009  2027-01-15            2.147945
  902   310.0       8.70           0.125009  2027-01-15            2.147945
  
  [903 rows x 5 columns],
  'puts':      strike  lastPrice  impliedVo

In [16]:
option_chains["MSFT"]["calls"]

,strike,lastPrice,impliedVolatility,expiration,time_to_expiration
0,240.0,186.69,0.000010,2024-11-29,0.019178
1,295.0,119.72,0.000010,2024-11-29,0.019178
2,300.0,113.75,0.000010,2024-11-29,0.019178
3,310.0,105.35,0.000010,2024-11-29,0.019178
4,320.0,95.01,0.000010,2024-11-29,0.019178
...,...,...,...,...,...
1447,610.0,18.00,0.062509,2027-01-15,2.147945
1448,620.0,17.51,0.062509,2027-01-15,2.147945
1449,630.0,15.99,0.062509,2027-01-15,2.147945
1450,640.0,14.27,0.062509,2027-01-15,2.147945


| **Option Type**  | **Underlying Price vs. Strike Price** | **Moneyness**          |
|-------------------|---------------------------------------|------------------------|
| **Call Option**   | Asset Price > Strike Price            | In the Money (ITM)     |
|                   | Asset Price = Strike Price            | At the Money (ATM)     |
|                   | Asset Price < Strike Price            | Out of the Money (OTM) |
| **Put Option**    | Asset Price < Strike Price            | In the Money (ITM)     |
|                   | Asset Price = Strike Price            | At the Money (ATM)     |
|                   | Asset Price > Strike Price            | Out of the Money (OTM) |


In [17]:
filtered_options = {}

lower_bound = 0.1  # Example: 0.1 years (roughly 36.5 days)
upper_bound = 2.0  # Example: 2 years
tolerance = 0.2    # Example: ±20% around the spot price

for ticker, data in option_chains.items():
    try:
        spot_price = data['spot_price']  # Get the spot price for the ticker
        
        for mode in ['calls', 'puts']:  # Iterate through calls and puts
            options_df = data[mode]  # Get the DataFrame for this mode
            
            # Apply filtering criteria
            filtered = options_df[
                (options_df["time_to_expiration"] >= lower_bound) &
                (options_df["time_to_expiration"] <= upper_bound) &
                (options_df["strike"] >= spot_price * (1 - tolerance)) &
                (options_df["strike"] <= spot_price * (1 + tolerance))
            ]
            
            # Ensure the ticker exists in the filtered options dictionary
            if ticker not in filtered_options:
                filtered_options[ticker] = {"spot_price": spot_price}
            
            # Store the filtered options
            filtered_options[ticker][mode] = filtered
            
            # Print filtered options for verification
            print(f"Filtered {mode} for {ticker}:\n", filtered)
    except Exception as e:
        print(f"Error processing {ticker}: {e}")


Filtered calls for AMD:
      strike  lastPrice  impliedVolatility  expiration  time_to_expiration
293   115.0      25.50           0.000010  2025-01-17            0.153425
294   120.0      20.10           0.000010  2025-01-17            0.153425
295   125.0      17.20           0.000010  2025-01-17            0.153425
296   130.0      13.88           0.000010  2025-01-17            0.153425
297   135.0      10.90           0.000010  2025-01-17            0.153425
..      ...        ...                ...         ...                 ...
770   145.0      31.70           0.007822  2026-06-18            1.569863
771   150.0      29.50           0.015635  2026-06-18            1.569863
772   155.0      27.40           0.015635  2026-06-18            1.569863
773   160.0      26.65           0.031260  2026-06-18            1.569863
774   165.0      25.50           0.031260  2026-06-18            1.569863

[135 rows x 5 columns]
Filtered puts for AMD:
      strike  lastPrice  impliedVolatili

Filtered calls for LRCX:
       strike  lastPrice  impliedVolatility  expiration  time_to_expiration
406     57.0      18.81           0.000010  2025-01-17            0.153425
407     58.0      13.40           0.000010  2025-01-17            0.153425
408     59.0      12.30           0.000010  2025-01-17            0.153425
409     60.0      10.60           0.000010  2025-01-17            0.153425
410     61.0       9.95           0.000010  2025-01-17            0.153425
...      ...        ...                ...         ...                 ...
1300    76.0      12.00           0.015635  2026-01-16            1.150685
1301    78.0      10.58           0.031260  2026-01-16            1.150685
1302    80.0       9.15           0.031260  2026-01-16            1.150685
1303    82.0       8.75           0.031260  2026-01-16            1.150685
1304    84.0       7.80           0.031260  2026-01-16            1.150685

[170 rows x 5 columns]
Filtered puts for LRCX:
       strike  lastPrice  

In [18]:
test = f.black_scholes_call(S = filtered_options, K, T, r, sigma)

SyntaxError: positional argument follows keyword argument (3585140701.py, line 1)

In [11]:
companies

,symbol,address1,city,state,zip,country,phone,website,industry,industryKey,...,sector,sectorKey,sectorDisp,longBusinessSummary,fullTimeEmployees,companyOfficers,irWebsite,maxAge,address2,fax
0,MMM,3M Center,Saint Paul,MN,55144-1000,United States,651 733 1110,https://www.3m.com,Conglomerates,conglomerates,...,Industrials,industrials,Industrials,3M Company provides diversified technology ser...,85000,[],http://phx.corporate-ir.net/phoenix.zhtml?c=80...,86400,NaN,NaN
1,AOS,11270 West Park Place,Milwaukee,WI,53224-9508,United States,414 359 4000,https://www.aosmith.com,Specialty Industrial Machinery,specialty-industrial-machinery,...,Industrials,industrials,Industrials,A. O. Smith Corporation manufactures and marke...,12000,[],http://investor.shareholder.com/aosmith/,86400,Suite 170 PO Box 245008,NaN
2,ABT,100 Abbott Park Road,North Chicago,IL,60064-6400,United States,224 667 6100,https://www.abbott.com,Medical Devices,medical-devices,...,Healthcare,healthcare,Healthcare,"Abbott Laboratories, together with its subsidi...",114000,[],http://www.abbottinvestor.com/phoenix.zhtml?c=...,86400,Abbott Park,NaN
3,ABBV,1 North Waukegan Road,North Chicago,IL,60064-6400,United States,847 932 7900,https://www.abbvie.com,Drug Manufacturers - General,drug-manufacturers-general,...,Healthcare,healthcare,Healthcare,"AbbVie Inc. discovers, develops, manufactures,...",50000,[],NaN,86400,NaN,NaN
4,ACN,1 Grand Canal Square,Dublin,NaN,D02 P820,Ireland,353 1 646 2000,https://www.accenture.com,Information Technology Services,information-technology-services,...,Technology,technology,Technology,Accenture plc provides strategy and consulting...,774000,[],http://investor.accenture.com/phoenix.zhtml?c=...,86400,Grand Canal Harbour,353 1 646 2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,XYL,300 Water Street SE,Washington,DC,20003,United States,202 869 9150,https://www.xylem.com,Specialty Industrial Machinery,specialty-industrial-machinery,...,Industrials,industrials,Industrials,"Xylem Inc., together with its subsidiaries, en...",23000,[],NaN,86400,Suite 200,NaN
499,YUM,1441 Gardiner Lane,Louisville,KY,40213,United States,502 874 8300,https://www.yum.com,Restaurants,restaurants,...,Consumer Cyclical,consumer-cyclical,Consumer Cyclical,"Yum! Brands, Inc., together with its subsidiar...",35000,[],http://www.yum.com/investors/,86400,NaN,NaN
500,ZBRA,3 Overlook Point,Lincolnshire,IL,60069,United States,847 634 6700,https://www.zebra.com,Communication Equipment,communication-equipment,...,Technology,technology,Technology,"Zebra Technologies Corporation, together with ...",9750,[],http://www.zebra.com/id/zebra/na/en/index/abou...,86400,NaN,NaN
501,ZBH,345 East Main Street,Warsaw,IN,46580,United States,574 373 3333,https://www.zimmerbiomet.com,Medical Devices,medical-devices,...,Healthcare,healthcare,Healthcare,"Zimmer Biomet Holdings, Inc., together with it...",18000,[],http://investor.zimmer.com/,86400,NaN,NaN


In [12]:
filtered_companies

,AMD,GOOGL,GOOG,AMZN,AAPL,T,BAC,BA,BMY,AVGO,...,PLTR,PYPL,PFE,PCG,SLB,TSLA,UBER,VZ,WMT,WFC
Date,,,,,,,,,,,,,,,,,,,,,
2022-11-16 00:00:00-05:00,72.699997,98.605835,98.747490,97.120003,147.241608,16.704006,35.322781,172.690002,69.990387,49.091938,...,8.060000,87.040001,43.316650,14.696905,50.856331,186.919998,30.040001,33.083588,48.104252,43.653851
2022-11-17 00:00:00-05:00,73.900002,98.117043,98.258690,94.849998,149.151550,16.721657,35.143139,172.779999,70.918694,49.128395,...,7.620000,85.639999,43.569069,14.547243,50.702778,183.169998,28.879999,33.328133,47.942276,43.446018
2022-11-18 00:00:00-05:00,73.570000,97.189346,97.560410,94.139999,149.715591,16.739302,35.162045,173.889999,71.185226,49.420990,...,7.390000,84.919998,43.478916,14.986255,51.288212,180.190002,28.959999,33.668747,48.664661,43.927807
2022-11-21 00:00:00-05:00,72.459999,95.363861,95.595238,92.459999,146.469711,16.624592,35.275505,172.940002,72.573090,49.639721,...,7.190000,80.629997,43.433842,14.956321,50.165329,167.869995,28.250000,33.729889,48.962677,44.248993
2022-11-22 00:00:00-05:00,75.250000,96.810287,97.091560,93.199997,148.617142,16.765778,35.445694,172.500000,72.481171,50.849438,...,7.220000,79.910004,44.245186,15.086031,51.480156,169.910004,28.080000,34.236446,49.137604,44.504066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-11 00:00:00-05:00,147.350006,180.350006,181.970001,206.839996,224.229996,22.309999,46.080002,148.960007,59.820000,178.910004,...,60.240002,86.940002,26.240000,21.090000,44.290001,350.000000,71.650002,40.439999,84.209999,72.559998
2024-11-12 00:00:00-05:00,143.630005,181.619995,183.320007,208.910004,224.229996,22.150000,45.860001,145.169998,59.020000,176.220001,...,59.849998,86.400002,26.190001,21.190001,44.330002,328.489990,71.370003,40.400002,84.989998,72.599998
2024-11-13 00:00:00-05:00,139.300003,178.880005,180.490005,214.100006,225.119995,22.299999,45.869999,139.970001,58.410000,173.580002,...,60.700001,87.309998,26.719999,21.280001,43.590000,330.239990,71.160004,41.139999,85.500000,72.769997


In [13]:
testing = yf.Ticker("MSFT").history(period="1d")
testing

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-11-20 00:00:00-05:00,416.869995,417.290009,410.579987,415.48999,19166200,0.0,0.0
